## MACS 30267 Final
Maha Adil, Angel Westbrook, Jason Gordon

[Intro to final]

In [1]:
import pandas as pd 

sentiment = pd.read_csv("Data/nltk_split.csv")

In [2]:
sentiment.info()
# Not many non-null for coordinates. Should prolly kill that. Also not sure what the unnamed ones are.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163669 entries, 0 to 163668
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0.2     163669 non-null  int64  
 1   Unnamed: 0.1     163669 non-null  int64  
 2   Unnamed: 0       163669 non-null  int64  
 3   id               163669 non-null  float64
 4   text             163669 non-null  object 
 5   favorite_count   163669 non-null  int64  
 6   retweet_count    163669 non-null  int64  
 7   created_at       163669 non-null  object 
 8   coordinates      43 non-null      object 
 9   score            163669 non-null  object 
 10  neg              163669 non-null  float64
 11  neu              163669 non-null  float64
 12  pos              163669 non-null  float64
 13  compound         163669 non-null  float64
 14  created_at_date  163669 non-null  object 
 15  Unnamed: 15      0 non-null       float64
 16  Unnamed: 16      0 non-null       floa

In [3]:
data = sentiment[["id", "text", "favorite_count", "retweet_count", "created_at", "coordinates", "score", "neg", "neu", "pos", "compound", "created_at_date"]]
data

,id,text,favorite_count,retweet_count,created_at,coordinates,score,neg,neu,pos,compound,created_at_date
0,1.030000e+18,Gotta love the facts. https://t.co/bZ2G8AZuo9,0,0,2018-08-13 10:40:21+00:00,NaN,"{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...",0.000,0.488,0.512,0.6369,8/13/2018
1,1.030000e+18,RT @ToolangiForest: A great day of action for ...,0,35,2018-08-13 10:40:10+00:00,NaN,"{'neg': 0.0, 'neu': 0.651, 'pos': 0.349, 'comp...",0.000,0.651,0.349,0.8805,8/13/2018
2,1.030000e+18,@jonkudelka Harvey Norman reckons climate chan...,2,0,2018-08-13 10:40:43+00:00,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,8/13/2018
3,1.030000e+18,RT @jayrosen_nyu: Why does skepticism about im...,0,52,2018-08-13 10:40:43+00:00,NaN,"{'neg': 0.146, 'neu': 0.62, 'pos': 0.234, 'com...",0.146,0.620,0.234,0.5267,8/13/2018
4,1.030000e+18,RT @FranceinIreland: On 5th November we call a...,0,16,2018-08-13 10:41:58+00:00,NaN,"{'neg': 0.095, 'neu': 0.657, 'pos': 0.248, 'co...",0.095,0.657,0.248,0.4939,8/13/2018
...,...,...,...,...,...,...,...,...,...,...,...,...
163664,9.220000e+17,#RRN https://t.co/9DTuWYbyzn E.P.A. Cancels Ta...,0,0,2017-10-23 12:31:37+00:00,NaN,"{'neg': 0.16, 'neu': 0.84, 'pos': 0.0, 'compou...",0.160,0.840,0.000,-0.2263,10/23/2017
163665,9.220000e+17,RT @TIME: The EPA has pulled its scientists fr...,0,78,2017-10-23 12:50:34+00:00,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,10/23/2017
163666,9.220000e+17,RT @cathmckenna: We must all take action on #C...,0,21,2017-10-23 15:10:36+00:00,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,10/23/2017
163667,9.220000e+17,RT @TIME: The EPA has pulled its scientists fr...,0,125,2017-10-23 15:17:57+00:00,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,10/23/2017


## RQ2: Can we classify sentiment from ML techniques?

Idea: use the compound score (ranges from -1 to 1), and see if we can utilize the text column to predict in a classification pipeline.

In [4]:
# Load Libraries
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

In [2]:
#%pip install textmining
#%pip install nltk

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
y = data["compound"]
X = data.drop(["pos", "neg", "neu", "compound", "score"], axis=1)
X.head()

In [ ]:
sns.displot(y)

In [ ]:
np.random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train)

In [ ]:
## Need to set up Linear regression
lin_reg = LinearRegression()
lin_model = lin_reg.fit(X_train, np.ravel(y_train))
y_pred = lin_reg.predict(X_validate)
